### Your name:

<pre> Zihan Zhao</pre>


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
print("hello world!")

hello world!


Open the housing data


In [7]:
import os
import tarfile

HOUSING_PATH = "."

def fetch_housing_data(housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

np.random.seed(42)
housing = load_housing_data()
housing_label = housing.loc[:,"median_house_value"]
housing = housing.drop("median_house_value",axis=1)
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY


#### Considerations for building pipeline:

- Make your notebook as compact as possible. 
- Split data into training and testing sets below.
- Convert all categorical data to one-hot vectors below
- Normalize all non-categorical data 
-  Perform KNN regression using a variety of values for n_neighbors (K) between 1 and 10 and both "uniform" and "distance" weights via a grid search where  *housing_labels* is the output and all other features are the input (similar to as seen in lecture two.)

### Build full pipeline for the data analysis following the example of the notebook.
 Hint: the main part requested to change is the algorithm used (KNN regression)


In [20]:
## Building Pipeline to prepare data:
housing_num = housing.select_dtypes(include=[np.number])
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
try:
    from sklearn.impute import SimpleImputer # Scikit-Learn 0.20+
except ImportError:
    from sklearn.preprocessing import Imputer as SimpleImputer
try:
    from sklearn.preprocessing import OrdinalEncoder
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20
imputer = SimpleImputer(strategy="median")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

##full pipeline
from future_encoders import ColumnTransformer # Scikit-Learn < 0.20
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        # Convert all categorical data to one-hot vectors below
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [24]:
##Splitting data
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(housing_prepared, housing_label, test_size=0.2, random_state=42)

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
param_grid = dict(n_neighbors=range(1,10),weights=['uniform','distance'])
grid = GridSearchCV(knn,param_grid,cv=10)
grid.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': range(1, 10), 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
print("Best_params:",grid.best_params_)

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

best = grid.best_estimator_
# CV result
def display_scores(scores):
    print("###  CV Result    ###")
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

knn_scores = cross_val_score(best, x_train, y_train,
                                scoring="neg_mean_squared_error", cv=10)
knn_rmse_scores = np.sqrt(-knn_scores)
display_scores(knn_rmse_scores)

# Estimation for train with best model
housing_predictions = best.predict(x_train)
knn_mse = mean_squared_error(y_train, housing_predictions)
knn_rmse = np.sqrt(knn_mse)

# Estimation for test
test_predictions = best.predict(x_test)
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_rmse
print("Train RMSE: %.3f \n Test RMSE: %.3f" %(knn_rmse, test_rmse) )

Best_params: {'n_neighbors': 9, 'weights': 'distance'}
###  CV Result    ###
Mean: 59644.72465396549
Standard deviation: 2005.152605072303
Train RMSE: 0.000 
 Test RMSE: 60265.938


### Conclusions
For what values of n_neighbors and weight does KNeighborsRegressor perform the best? Does it perform as well on the housing data as the linear regressor from the lectures? Why do you think this is?

<pre> KNeighborsRegressor with 9 neighbors and weight with 'distance' has best performance; KNeighborsRegressor performs better than linear regressor in test data.

In both train and test dataset, the RMSE of KNN is smaller than the RMSE of linear regressor, which means that KNN has better performance in both in-sample and out-of-sample data. Also, Mean and standard deviation of KNN in Cross Validation indicate that KNN has better performance and more stable than linear regressor. 

There might be several reasons why KNN has better performance thant linear regression in this dataset. Firstly, We can consider that the variables may have non-linear relationships with target variable. Secondly, the KNN model is tuned with Gridsearch method with manny different patterns, the selected model has the best performance among all proposed models when linear regressor has no any tuning parameters, which means we can not tune any parameter for linear regressor when using cross validation. </pre>

